In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F306%2F666%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20251204%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20251204T104202Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D89b9331356a86e3578fbe4b8b50046f8ef9cb7c5922848809a6b8b317d5c12288b48f4ef230c753575c5f93fb353a98e8a2f06a00501a9c2fff7f2645913898268505c22ca15fdee1518efdc8d874b397f073682c1cc29954949c384d6500d0204c5a92db9ed6531d40e33a57f62ff1599436aa25cffa44b277adf0ae13efaa91586e97a331557b929ab1f1dbf88a47466f53d743daedd42cb2a270b479c923c85880a0b5f08954c2bc9f654af55b9f6a66f98b4075967cf81888003a29adfb67d2fd9df5a2c78b2799bdb745511167366704c9a3ca2e75aaf5480d2983741823b602991bc275b44dfa54019ec1e13cebd6986db2495c3a68f1b8f5c15dcc11f'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


character(0)

character(0)

character(0)

character(0)

character(0)

character(0)

[1] "Downloading and uncompressing: "
[1] "Downloaded and uncompressed: "
[1] "Data source import complete"


In [ ]:
# German credit Scoring

# INTRODUCTION
# ============

# Le projet est découpé en 3 parties
# Exploratoty data analysis (EDA)
# Feature engineering
# Model
# Prediction


# Load Packages
library('ggplot2') # visualization
library('ggthemes') # visualization
library('scales') # visualization
library('dplyr') # data manipulation
library('mice') # imputation
library('randomForest') # classification algorithm
library(gtools) # for discretisation
library(corrplot)
library(Hmisc)
library(devtools)
library(PerformanceAnalytics)
library(FactoMineR)



# Importing Data

data <- read.csv("..//input/UCI_Credit_Card.csv")

head(data)
summary(data)

# In R, Catégorical variables are called Factors

# Make variables factors into factors
factor_vars <- c('SEX','EDUCATION','MARRIAGE','default.payment.next.month')


data[factor_vars] <- lapply(data[factor_vars], function(x) as.factor(x))


# Boxplots amount of credit lim by education
# observations (points) are overlayed and jittered
qplot(EDUCATION, LIMIT_BAL, data=data, geom=c("boxplot", "jitter"),
      fill=EDUCATION, main="LIMIT_AMT by Education cat",
      xlab="", ylab="amount of given credit")




# FEATURE ENGINEERING

# We will looke at all our 25 variables, distributions, level(for factors)



# Lets look at Education variable
plot(data$EDUCATION)

# we remark that that levels (0 , 4 , 5 and 6 reprents a tini amount of the data)
x = data$EDUCATION[data$EDUCATION == c(0,4,5,6)]
data$EDUCATION[data$EDUCATION== 4]         <- 0
data$EDUCATION[data$EDUCATION== 5]  <- 0
data$EDUCATION[data$EDUCATION== 6]  <- 0

plot(data$EDUCATION)

ggplot(data, aes(x = EDUCATION, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Education') +
  theme_excel()
A = data
# From now we are using A instead of data

# Barplot
bp<- ggplot(A, aes(x=EDUCATION, fill = default.payment.next.month))+
  geom_bar(width = 1)+
  coord_polar()
qplot(AGE, data = A, geom = "density", fill = default.payment.next.month)
bp
# Thats better


# LETS LOOK AT LIMIT_AMOUNT
# We plot a histogram for limit amount (include individuals and family credits)

plot(A$LIMIT_BAL)
# Thats better!

mean(A$LIMIT_BAL)

# 167484.3 $
boxplot(x = data[,2],col = "red")


# let's get a look to age variable

hist(A$AGE,col = "lightblue",freq  = 2,main = "histogram & distrubution of client's age",breaks = 10,xlab = "Age of the client", ylab = "Frequency")

ggplot(A, aes(x = AGE, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Age') +
  theme_excel()


# exploring limit_bal payment

hist(data$LIMIT_BAL, breaks = 20) # 10K to 1G , mean = 167484

# RELATION BTWN DEFAULT AND EDUCATION

ggplot(data, aes(x = EDUCATION, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Education') +
  theme_excel()


# RELATION BTWN DEFAULT AND SEX

ggplot(data, aes(x = SEX, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Sex') +
  theme_economist_white()


# RELATION BTWN DEFAULT AND MARIAGE

ggplot(data, aes(x = MARRIAGE, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Mariage') +
  theme_excel()



# RELATION BTWN LIMIT_BAL AND EDUCATION
ggplot(data, aes(x = EDUCATION, fill = LIMIT_BAL)) +
  geom_bar() +
  labs(x = 'Education') +
  labs(y = 'Limit crédit') +
  theme_few()

# relation btwn age and default

ggplot(data, aes(AGE, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 6) +
  # I include education since we know (a priori) it's a significant predictor
  facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

# we see clearly an important rate of default in the university category

# lets explore PAY_0

ggplot(data, aes(PAY_0, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_excel()

# we see that if pay_0 is btwn 1 and 6, the probability of default is a lot highter


# now let's see pay_2
ggplot(data, aes(PAY_2, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

ggplot(data, aes(PAY_3, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

ggplot(data, aes(PAY_4, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()







#DIMENTIONZLITY REDUCTION

# Principal component analysis (PCA) is used to summarize the information in a data set described by multiple variables.
# PCA reduces the dimensionality of data containing a large set of variables. This is achieved by transforming the initial variables into a new small set of variables without loosing the most important information in the original data set.

# pca applies only to quantitative features
# we are going to explore the correlations between our Bill_amt 1 to6 , pay_amt 1 to 6 and limit_bal

df <- A[,c(2,13,14,15,16,17,18,19,20,21,22,23,24)]
head(df)




res.pca <- PCA(df, scale.unit = TRUE, graph = TRUE)



# visualisation of the correlation matrix


chart.Correlation(df, histogram=TRUE, pch=19)


print(res.pca)


head(res.pca$var$coord)

# Cos2 : quality of the representation for variables on the factor map
head(res.pca$var$cos2)


# Contributions of the variables to the principal components

# The contributions of variables in accounting for the variability in a given principal component are (in percentage) : (variable.cos2 * 100) / (total cos2 of the component)


head(res.pca$var$contrib)

# The larger the value of the contribution, the more the variable contributes to the component.


# The function dimdesc()[in FactoMineR] can be used to identify the most correlated variables with a given principal component.


dimdesc(res.pca, axes = 1:3, proba = 0.05)

res.desc <- dimdesc(res.pca, axes = c(1,2))

res.desc

res.desc$Dim.1





# German credit Scoring

# INTRODUCTION
# ============

# Le projet est découpé en 3 parties
# Exploratoty data analysis (EDA)
# Feature engineering
# Model
# Prediction


# Load Packages
library('ggplot2') # visualization
library('ggthemes') # visualization
library('scales') # visualization
library('dplyr') # data manipulation
library('mice') # imputation
library('randomForest') # classification algorithm
library(gtools) # for discretisation
library(corrplot)
library(Hmisc)
library(devtools)
library(PerformanceAnalytics)
library(FactoMineR)



# Importing Data

data <- read.csv("..//input/UCI_Credit_Card.csv")

head(data)
summary(data)

# In R, Catégorical variables are called Factors

# Make variables factors into factors
factor_vars <- c('SEX','EDUCATION','MARRIAGE','default.payment.next.month')


data[factor_vars] <- lapply(data[factor_vars], function(x) as.factor(x))


# Boxplots amount of credit lim by education
# observations (points) are overlayed and jittered
qplot(EDUCATION, LIMIT_BAL, data=data, geom=c("boxplot", "jitter"),
      fill=EDUCATION, main="LIMIT_AMT by Education cat",
      xlab="", ylab="amount of given credit")




# FEATURE ENGINEERING

# We will looke at all our 25 variables, distributions, level(for factors)



# Lets look at Education variable
plot(data$EDUCATION)

# we remark that that levels (0 , 4 , 5 and 6 reprents a tini amount of the data)
x = data$EDUCATION[data$EDUCATION == c(0,4,5,6)]
data$EDUCATION[data$EDUCATION== 4]         <- 0
data$EDUCATION[data$EDUCATION== 5]  <- 0
data$EDUCATION[data$EDUCATION== 6]  <- 0

plot(data$EDUCATION)

ggplot(data, aes(x = EDUCATION, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Education') +
  theme_excel()
A = data
# From now we are using A instead of data

# Barplot
bp<- ggplot(A, aes(x=EDUCATION, fill = default.payment.next.month))+
  geom_bar(width = 1)+
  coord_polar()
qplot(AGE, data = A, geom = "density", fill = default.payment.next.month)
bp
# Thats better


# LETS LOOK AT LIMIT_AMOUNT
# We plot a histogram for limit amount (include individuals and family credits)

plot(A$LIMIT_BAL)
# Thats better!

mean(A$LIMIT_BAL)

# 167484.3 $
boxplot(x = data[,2],col = "red")


# let's get a look to age variable

hist(A$AGE,col = "lightblue",freq  = 2,main = "histogram & distrubution of client's age",breaks = 10,xlab = "Age of the client", ylab = "Frequency")

ggplot(A, aes(x = AGE, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Age') +
  theme_excel()


# exploring limit_bal payment

hist(data$LIMIT_BAL, breaks = 20) # 10K to 1G , mean = 167484

# RELATION BTWN DEFAULT AND EDUCATION

ggplot(data, aes(x = EDUCATION, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Education') +
  theme_excel()


# RELATION BTWN DEFAULT AND SEX

ggplot(data, aes(x = SEX, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Sex') +
  theme_economist_white()


# RELATION BTWN DEFAULT AND MARIAGE

ggplot(data, aes(x = MARRIAGE, fill = default.payment.next.month)) +
  geom_bar() +
  labs(x = 'Mariage') +
  theme_excel()



# RELATION BTWN LIMIT_BAL AND EDUCATION
ggplot(data, aes(x = EDUCATION, fill = LIMIT_BAL)) +
  geom_bar() +
  labs(x = 'Education') +
  labs(y = 'Limit crédit') +
  theme_few()

# relation btwn age and default

ggplot(data, aes(AGE, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 6) +
  # I include education since we know (a priori) it's a significant predictor
  facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

# we see clearly an important rate of default in the university category

# lets explore PAY_0

ggplot(data, aes(PAY_0, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_excel()

# we see that if pay_0 is btwn 1 and 6, the probability of default is a lot highter


# now let's see pay_2
ggplot(data, aes(PAY_2, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

ggplot(data, aes(PAY_3, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  # facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()

ggplot(data, aes(PAY_4, fill = default.payment.next.month)) +
  geom_histogram(binwidth = 1) +
  # I include education since we know (a priori) it's a significant predictor
  facet_grid(.~EDUCATION) +
  theme_fivethirtyeight()







#DIMENTIONZLITY REDUCTION

# Principal component analysis (PCA) is used to summarize the information in a data set described by multiple variables.
# PCA reduces the dimensionality of data containing a large set of variables. This is achieved by transforming the initial variables into a new small set of variables without loosing the most important information in the original data set.

# pca applies only to quantitative features
# we are going to explore the correlations between our Bill_amt 1 to6 , pay_amt 1 to 6 and limit_bal

df <- A[,c(2,13,14,15,16,17,18,19,20,21,22,23,24)]
head(df)




res.pca <- PCA(df, scale.unit = TRUE, graph = TRUE)



# visualisation of the correlation matrix


chart.Correlation(df, histogram=TRUE, pch=19)


print(res.pca)


head(res.pca$var$coord)

# Cos2 : quality of the representation for variables on the factor map
head(res.pca$var$cos2)


# Contributions of the variables to the principal components

# The contributions of variables in accounting for the variability in a given principal component are (in percentage) : (variable.cos2 * 100) / (total cos2 of the component)


head(res.pca$var$contrib)

# The larger the value of the contribution, the more the variable contributes to the component.


# The function dimdesc()[in FactoMineR] can be used to identify the most correlated variables with a given principal component.


dimdesc(res.pca, axes = 1:3, proba = 0.05)

res.desc <- dimdesc(res.pca, axes = c(1,2))

res.desc

res.desc$Dim.1





